In [26]:
from odoo import api, fields, models

location = self.env['stock.location'].search([('name', '=', 'MAG/Stock')], limit=1)
products_to_notify = []

products_below_threshold = self.env['product.template'].search([
    ('qty_available', '<=', 15),  # Adjust the threshold as needed
    ('location_id', '=', location.id),
])

for product in products_below_threshold:
    # Only add products with quantity greater than zero
    if product.qty_available > 0:
        products_to_notify.append({
            'name': product.name,
            'qty_available': product.qty_available,
        })

# Check if there are products to notify
if products_to_notify:
    # Use with_delay to enqueue the job asynchronously
    print('notifications sent')
    print("Sending notification for products that need restocking.")
    channel = self.env['mail.channel'].channel_get([self.env.user.partner_id.id])
    channel_id = self.env['mail.channel'].browse(channel["id"])

    message_body = "The following products are below the restocking threshold:\n"
    for product_info in products_to_notify[:10]:
        message_body += f"Product: {product_info['name']}, Quantity: {product_info['qty_available']}\n"
    print(message_body)


notifications sent
Sending notification for products that need restocking.
The following products are below the restocking threshold:
Product: 
ST701F MICROWAVE MOTION SENSOR RECESSED FIXTURE 180°, Quantity: 3.0
Product:  1TB Hard Drive - 2.5" Internal - SATA (SATA/600) - 5400rpm Seagate BarraCuda ST1000LM048, Quantity: 4.0
Product:  900 PIÈCE D'ANGLE, Quantity: 8.0
Product:  ROLINE HDMI High Speed Cable 50 m with Ethernet, M/M - Repeater, Quantity: 4.0
Product:  TP-Link UB500 Bluetooth 5.0 Adapter for Desktop Computer/Notebook - USB 2.0, Quantity: 1.0
Product: ( EMG-330P + DMS-1) Microphone directionnel electret miniature avec col de cygne + Pied de micro de table, Quantity: 3.0
Product: 0,5 meters N-Male to RP-SMA Male Pigtail Cable 2.4GHz, Quantity: 7.0
Product: 0T0/4UA/8Z MIX CARTE 4XNUMERIQUE - 8XANALOGIQUE, Quantity: 3.0
Product: 100 x AM Tag  HD2105 Type stylo avec attache cable 175MM-16MM, Quantity: 10.0
Product: 12.99.1105 VALUE USB 3.2 Gen 1 to Gigabit Ethernet Converter, Qua

In [20]:
if products_to_notify:
    print(f"Processing product: {product.name}, Quantity: {product.qty_available}")
    # Use with_delay to enqueue the job asynchronously  # Limit to the first 10 elements
    print('Notifications sent')

Processing product: Ø 40 - 3m CONDUIT RIGIDE PVC NU GRIS CLAIR, Quantity: 0.0
Notifications sent


In [10]:
product_ids = self.env['product.product'].search([]).ids
batch_size = 500
for i in range(0, len(product_ids), batch_size):
    batch_product_ids = product_ids[i:i+batch_size]
    _check_stock_and_notify_batch(batch_product_ids)


TypeError: _check_stock_and_notify_batch() missing 1 required positional argument: 'product_ids'

In [5]:
def _check_stock_and_notify_batch(self, product_ids):
        _logger.info(f"Processing batch of products: {product_ids}")        
        try:
            low_stock_products = self.env['product.product'].read_group(
                [('id', 'in', product_ids), ('qty_available', '<=', self.threshold)],
                ['qty_available'],
                ['default_code'],  # Use the actual field name of the Many2one field in the model
                lazy=False
            )
    
            for product_data in low_stock_products:
                product_id = product_data['id'][0]
                qty_available = product_data.get('qty_available', 0)
    
                # Check and notify logic
                if qty_available <= self.threshold:
                    product = self.env['product.product'].browse(product_id)
                    _logger.info(f"Low stock detected for product '{product.name}' ({qty_available} units)")
                    # Customize the notification method according to your needs
                    # Example: You can use the 'mail' module to send an email.
                    self.env['mail.thread'].message_post(
                        body=f"Le produit '{product.name}' est faible en stock , seulement ({product.qty_available} pieces restants!).",
                        subject='Low Stock Notification',
                        subtype='mail.mt_comment',
                        partner_ids=[(4, user.id) for user in product.message_follower_ids],
                    )
            _logger.info(f"Finished processing batch of products: {product_ids}")
            
        except Exception as e:
            _logger.error(f"Error during batch processing: {e}")
            _logger.info("After processing batch")